# Analysis of longitudinal Alexa ranks of partisan websites to find their popularity trends

In [1]:

import json
import os
import glob
import time
import datetime
from email.Utils import formatdate
import pandas as pd
dir_name = 'data/Alexa_rank_charts'
json_pattern = os.path.join(dir_name,'*.json')
file_list = glob.glob(json_pattern)

In [2]:
def get_formatted_list(rank_list):
    for each_dict in rank_list:
        for key, val in each_dict.items():
            if key == 'x':
                val = time.strftime("%a %d %b %Y %H:%M:%S GMT", time.gmtime(float(val) / 1000.0))
                each_dict[key] = val
    return rank_list

In [36]:
def get_month_rank_dict(rank_list):
    month_rank_dict = {}
    for each_dict in rank_list:
        for key, val in each_dict.items():
            if key == 'x':
                if 'Jun 2016' in val or 'July' in val or 'Aug' in val or 'Sep' in val or 'Oct'in val:
                    if month_rank_dict.has_key('Before Nov 2016') == False:
                        month_rank_dict['Before Nov 2016'] = []
                        month_rank_dict['Before Nov 2016'].append(each_dict['y'])
                    else:
                        month_rank_dict['Before Nov 2016'].append(each_dict['y'])
                if 'Nov' in val or 'Dec' in val or 'Jan' in val or 'Feb' in val:
                    if month_rank_dict.has_key('Nov-Feb') == False:
                        month_rank_dict['Nov-Feb'] = []
                        month_rank_dict['Nov-Feb'].append(each_dict['y'])
                    else:
                        month_rank_dict['Nov-Feb'].append(each_dict['y'])
                if 'Mar' in val or 'Apr' in val or 'May' in val or 'Jun 2017' in val:
                    if month_rank_dict.has_key('After Feb 2017') == False:
                        month_rank_dict['After Feb 2017'] = []
                        month_rank_dict['After Feb 2017'].append(each_dict['y'])
                    else:
                        month_rank_dict['After Feb 2017'].append(each_dict['y'])
    return month_rank_dict


In [37]:
rank_dict = {}

for each_file in file_list:
    json_data = open(each_file).read()
    try:
        data = json.loads(json_data)
        url = data['chartData'].keys()[0]
        #print url
        rank_list = get_formatted_list(data['chartData'][url]['rank'])
        rank_dict[url] = rank_list
    except ValueError, e:
        print e, each_file
        

In [38]:
directory = "data/Temporal_data/"

for key in rank_dict.keys():
    each_df = pd.DataFrame(rank_dict[key])
    filename = directory + key + ".csv"
    each_df.to_csv(filename, index=False, encoding='utf-8')


In [39]:
month_rank_dict = {}
for key, val in rank_dict.items():
    month_rank_dict[key] = get_month_rank_dict(val)
no_rank_list = []
for key, val in month_rank_dict.items():
    if val == {}:
        no_rank_list.append(key)

In [40]:
def get_top_rank_dict(month_dict):
    top_rank_dict = {}
    for key, val in month_dict.items():
        top_rank_dict[key] = min(val)
    return top_rank_dict
    

In [41]:
import operator
from partisan_sites_parser import *

top_rank_dict = {}
max_rank_dict = {}
for key, val in month_rank_dict.items():
    top_rank_dict[key] = get_top_rank_dict(val)

In [43]:
len(top_rank_dict)

586

In [44]:
no_rank_list = []
for key, val in top_rank_dict.items():
    #print val
    if val == {}:
        top_rank_dict.pop(key)
        no_rank_list.append(key)

In [46]:
len(left)

179

In [47]:
len(right)

489

In [48]:
rank_chart_left = []
rank_chart_right = []
from partisan_sites_parser import *
for each_key in top_rank_dict.keys():
    if each_key in left:
        rank_chart_left.append(each_key)
    if each_key in right:
        rank_chart_right.append(each_key)

In [49]:
len(rank_chart_right)

389

In [50]:
len(rank_chart_left)

159

In [52]:
def get_max_rank_dict(rank_chart_dict):
    max_rank_dict = {}
    for key, val in rank_chart_dict.items():
        max_rank = min(val.values())
        if 'Nov-Feb' in val.keys():
            if max_rank == val['Nov-Feb']:
                if max_rank_dict.has_key('Nov-Feb') == False:
                    max_rank_dict['Nov-Feb'] = []
                    max_rank_dict['Nov-Feb'].append(key)
                else:
                    max_rank_dict['Nov-Feb'].append(key)
        if 'After Feb 2017' in val.keys():
            if max_rank == val['After Feb 2017']:
                if max_rank_dict.has_key('After Feb 2017') == False:
                    max_rank_dict['After Feb 2017'] = []
                    max_rank_dict['After Feb 2017'].append(key)
                else:
                    max_rank_dict['After Feb 2017'].append(key)
        if 'Before Nov 2016' in val.keys():
            if max_rank == val['Before Nov 2016']:
                if max_rank_dict.has_key('Before Nov 2016') == False:
                    max_rank_dict['Before Nov 2016'] = []
                    max_rank_dict['Before Nov 2016'].append(key)
                else:
                    max_rank_dict['Before Nov 2016'].append(key)
    return max_rank_dict

In [53]:
max_rank_dict = get_max_rank_dict(top_rank_dict)

In [54]:
max_rank_left = {}
max_rank_right = {}

In [55]:
for key, val in max_rank_dict.items():
    for each_val in val:
        if each_val in rank_chart_left:
            if key not in max_rank_left.keys():
                max_rank_left[key] = []
                max_rank_left[key].append(each_val)
            else:
                max_rank_left[key].append(each_val)
        if each_val in rank_chart_right:
            if key not in max_rank_right.keys():
                max_rank_right[key] = []
                max_rank_right[key].append(each_val)
            else:
                max_rank_right[key].append(each_val)

In [56]:
def get_max_rank_plot_data(max_rank_dict):
    max_rank_plot_data = {}
    for key, val in max_rank_dict.items():
        max_rank_plot_data[key] = len(set(val))
    return max_rank_plot_data

In [57]:
max_rank_plot_left = {}
max_rank_plot_right = {}

In [58]:
max_rank_plot_left = get_max_rank_plot_data(max_rank_left)
max_rank_plot_right = get_max_rank_plot_data(max_rank_right)

In [59]:
max_rank_plot_left

{'After Feb 2017': 40, 'Before Nov 2016': 54, 'Nov-Feb': 65}

In [60]:
max_rank_plot_right

{'After Feb 2017': 86, 'Before Nov 2016': 116, 'Nov-Feb': 188}

In [ ]:
election_websites = {}

for key, val in top_rank_dict.items():
    if len(val.values()) > 5:
        max_rank = min(val.values())
        if max_rank == val['Nov'] or max_rank == val['Dec'] or max_rank == val['Jan']:
            delta_before = val['Before'] - max_rank
            delta_after = max_rank - val['After']
            election_websites[key] = {}
            election_websites[key]['max_rank'] = max_rank
            election_websites[key]['delta_before'] = delta_before
            election_websites[key]['delta_after'] = delta_after

In [ ]:
import pandas as pd
df = pd.DataFrame(election_websites)
